In [18]:
# Import packages used for the analysis
import os
from glob import glob

import earthpy as et
import earthpy.appeears as eaapp
import geopandas as gpd
import requests
import geoviews as gv
import holoviews as hv
import matplotlib.pyplot as plt
import rioxarray as rxr
import xarray as xr

In [2]:
grasslands_url = (
    "https://data.fs.usda.gov/geodata/edw/edw_resources/shp"
    "/S_USA.NationalGrassland.zip")
grasslands_gdf = gpd.read_file(grasslands_url)
grasslands_gdf

,NATIONALGR,GRASSLANDN,GIS_ACRES,SHAPE_AREA,SHAPE_LEN,geometry
0,295509010328,Sheyenne National Grassland,70428.175,0.033356,4.097398,"MULTIPOLYGON (((-97.31081 46.51457, -97.30559 ..."
1,295520010328,Little Missouri National Grassland,1025313.868,0.492746,60.420702,"MULTIPOLYGON (((-104.01688 47.51793, -104.0115..."
2,295513010328,Thunder Basin National Grassland,626249.208,0.282888,44.088050,"MULTIPOLYGON (((-105.46005 43.31908, -105.4601..."
3,295521010328,Oglala National Grassland,215804.927,0.096279,1.970612,"POLYGON ((-103.72477 43.00100, -103.72007 43.0..."
4,295519010328,Crooked River National Grassland,173593.026,0.079499,2.815873,"POLYGON ((-120.98025 44.72935, -120.98030 44.7..."
5,295515010328,Caddo National Grassland,68479.549,0.026940,1.159342,"MULTIPOLYGON (((-95.85492 33.79814, -95.85494 ..."
6,295523010328,Pawnee National Grassland,208424.885,0.089972,15.341594,"MULTIPOLYGON (((-104.58106 40.82664, -104.5810..."
7,295522010328,Comanche National Grassland,444413.904,0.183064,26.658022,"MULTIPOLYGON (((-104.02263 37.69224, -104.0225..."
8,295524010328,Cimarron National Grassland,109101.348,0.044765,5.539623,"MULTIPOLYGON (((-101.98977 37.11933, -101.9852..."
9,295508010328,Kiowa National Grassland,144281.321,0.058543,9.858642,"MULTIPOLYGON (((-104.30414 36.08063, -104.3041..."


In [3]:
# Index of /POLARIS/PROPERTIES/v1.0/ph/mean/60_100
# Change it according to chosen grassland units 
# (unit might not fit into one tile: 
# load in more than one tile per grassland 
# and need to merge with rioxarray (greenspace assignment)
# Download data and load it back again (ndvi assignment)

min_lat, min_lon = 26, -99
max_lat, max_lon = min_lat+1, min_lon+1
polaris_url_format = (
    "http://hydrology.cee.duke.edu/POLARIS/PROPERTIES/v1.0"
    "/ph/mean/60_100/lat{min_lat}{max_lat}_lon{min_lon}{max_lon}.tif"
)

polaris_url = polaris_url_format.format(
    min_lat=min_lat, min_lon=min_lon, max_lat=max_lat, max_lon=max_lon)

rxr.open_rasterio(polaris_url, masked=True).squeeze()

# check crs

<xarray.DataArray (y: 3600, x: 3600)>
[12960000 values with dtype=float32]
Coordinates:
    band         int32 1
  * x            (x) float64 -99.0 -99.0 -99.0 -99.0 ... -98.0 -98.0 -98.0 -98.0
  * y            (y) float64 27.0 27.0 27.0 27.0 27.0 ... 26.0 26.0 26.0 26.0
    spatial_ref  int32 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0

In [14]:
# https://lpdaac.usgs.gov/products/srtmgl1v003/
# Elevation data

cedar_river_gdf = (
    grasslands_gdf
    .set_index('GRASSLANDN')
    .loc[['Cedar River National Grassland']]
)

# might be the looping variable
download_key ='Cedar-River-SRTM'

srtm_downloader = eaapp.AppeearsDownloader(
    download_key = download_key,
    product='SRTMGL1_NC.003',
    layer='SRTMGL1_DEM',
    start_date='02-11-2000',
    end_date='02-21-2000',
    polygon=cedar_river_gdf)
srtm_downloader.download_files()

In [16]:
srtm_paths = glob(
    os.path.join(
        srtm_downloader.data_dir,
        'SRTMGL1_NC.003*',
        '*.tif')
)

[rxr.open_rasterio(srtm_path, masked=True).squeeze() for srtm_path in srtm_paths][0]

# check crs


<xarray.DataArray (y: 470, x: 2283)>
[1073010 values with dtype=float32]
Coordinates:
    band         int32 1
  * x            (x) float64 -101.9 -101.9 -101.9 ... -101.2 -101.2 -101.2
  * y            (y) float64 46.07 46.07 46.07 46.07 ... 45.95 45.94 45.94 45.94
    spatial_ref  int32 0
Attributes:
    add_offset:     0.0
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    units:          Meters

In [19]:
maca_url = (
    "http://thredds.northwestknowledge.net:8080/thredds/ncss"
    "/agg_macav2metdata_pr_CCSM4_r6i1p1_historical_1950_2005_CONUS_monthly.nc"
    "?disableLLSubset=on"
    "&disableProjSubset=on"
    "&horizStride=1"
    "&time_start=1980-01-15T00%3A00%3A00Z"
    "&time_end=1980-12-15T00%3A00%3A00Z"
    "&timeStride=1&accept=netcdf"
)

maca_response = requests.get(maca_url)

# # cotext magager
# with open('maca.nc', 'wb') as maca_file:
#     maca_file.write(maca_response.content)
# xr.open_dataset('maca.nc')